In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
#pd.set_option('display.max_rows', None)
import datetime

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
source_df = pd.read_csv('../input/order-brushing-dataset-shopee-code-league-week-1/order_brush_order.csv')
source_df['event_time'] = pd.to_datetime(source_df['event_time'])

In [ ]:
source_df.shape

In [ ]:
filt_shop = source_df.groupby(by=['shopid']).size()[source_df.groupby(by=['shopid']).size()>=3]
filt_shop_df = pd.DataFrame({'shopid': filt_shop.index,'jumlah_order':filt_shop.values})

shop_less_than_3 = source_df.groupby(by=['shopid']).size()[source_df.groupby(by=['shopid']).size()<3]
shop_less_than_3_df = pd.DataFrame({'shopid': shop_less_than_3.index,'jumlah_order':shop_less_than_3.values})
shop_less_than_3_df['userid'] = 0

In [ ]:
#Main function (Revised)
import timeit

source_df = source_df.sort_values(by='event_time') 
import datetime
results_df = pd.DataFrame({'shopid':[],'userid':[]}) #Create initial dataframe to tabulate the results
it = 0
start1 = timeit.default_timer()
for shop_id_it in filt_shop_df['shopid']:
    start = timeit.default_timer()
    it=it+1
#     if (it%100) ==0:
#         print("Iterasi ke-{} dari 2640".format(i))
    
    #Create dataframe with this shopid
    iter_shop_df = source_df.loc[source_df['shopid']==shop_id_it].sort_values(by='event_time') 
    #Find user(s) in this shopid which have equal or more than 3 orders
    filt_user = iter_shop_df.groupby(by=['userid']).size()[iter_shop_df.groupby(by=['userid']).size()>=3]
    filt_user_df = pd.DataFrame({'userid':filt_user.index,'jumlah_order':filt_user.values})
    if filt_user_df.shape[0] ==0: #Condition when there are no user with more than 3 orders
        results_df = results_df.append({'shopid':shop_id_it,'userid':0},ignore_index=True) 
    else:
        results_temp_df = pd.DataFrame({'shopid':[],'userid':[]}) 
        temp_shop_df = pd.DataFrame({'shopid':[],'userid':[]})
        temp_df = iter_shop_df[iter_shop_df['userid'].isin(filt_user_df['userid'])]
        min_time = temp_df.iloc[0]['event_time']-datetime.timedelta(hours=3)
        max_time = temp_df.iloc[-1]['event_time']+datetime.timedelta(hours=3)
        const_filter_shop_df = iter_shop_df.loc[(iter_shop_df['event_time']>min_time)&(iter_shop_df['event_time']<max_time)]
        previous_order_time = min_time
        for start_window in range(const_filter_shop_df.shape[0]-2):
            start_time = const_filter_shop_df.iloc[start_window]['event_time']
            for end_window in range(start_window+2,const_filter_shop_df.shape[0]):
                end_time = const_filter_shop_df.iloc[end_window]['event_time']
                if end_time-start_time > datetime.timedelta(hours=1):
                    break
              
                it_time_df = const_filter_shop_df.loc[(const_filter_shop_df['event_time']>=start_time)&(const_filter_shop_df['event_time']<=end_time)]
                if it_time_df['orderid'].nunique()/it_time_df['userid'].nunique() < 3:
                    break
                temp_shop_df = temp_shop_df.append(it_time_df,ignore_index=True)  
        if temp_shop_df.shape[0]!=0:
            temp_shop_df = temp_shop_df.drop_duplicates() 
            print('Cheat!')
            to_append = pd.DataFrame({'shopid': shop_id_it,'userid': temp_shop_df['userid'].mode().values})
            results_df = results_df.append(to_append,ignore_index=True)
            #results_df = results_df.append({'shopid': shop_id_it,'userid': temp_shop_df['userid'].mode().values},ignore_index=True)
            stop = timeit.default_timer()
            print("Iterasi ke-{} dari {}, membutuhkan waktu {}".format(it,filt_shop_df.shape[0],stop - start))
        else:
            results_df = results_df.append({'shopid':shop_id_it,'userid':0},ignore_index=True)
             
shop_less_than_3 = source_df.groupby(by=['shopid']).size()[source_df.groupby(by=['shopid']).size()<3]
shop_less_than_3_df = pd.DataFrame({'shopid': shop_less_than_3.index})
shop_less_than_3_df['userid'] = 0
results_df = results_df.append(shop_less_than_3_df,ignore_index='True')
stop1 = timeit.default_timer()
print("Semua membutuhkan waktu: {}".format(stop1-start1))
results_df = results_df.astype(int)

In [ ]:
#Post pro
results_akhir_df = pd.DataFrame({'shopid':[],'userid':[]})
results_akhir_df = results_akhir_df.astype('int')
for shop_it in results_df['shopid'].unique():
    #print(shop_it)
    temp_df = results_df.loc[results_df['shopid']==shop_it]
    if temp_df.shape[0] == 1:
        #print("Kondisi 1")
        results_akhir_df = results_akhir_df.append(temp_df,ignore_index=True)
    else:
        #print("Kondisi 2")
        for it in range(temp_df.shape[0]):
            if it == 0:
                user = str(temp_df.iloc[it]['userid'])
            else:
                user = user+str(temp_df.iloc[it]['userid'])
            if it < temp_df.shape[0]-1:
                user = user+'&'
        results_akhir_df = results_akhir_df.append({'shopid':shop_it,'userid':user},ignore_index=True)

In [ ]:
results_akhir_df.head()

In [ ]:
print("Sebanyak {} toko melakukan order brushing".format(sum(results_akhir_df['userid']!=0)))
print("Sebanyak {} toko tidak melakukan order brushing".format(sum(results_akhir_df['userid']==0)))

In [ ]:
##Data Visualization
##DATA VISUALIZATION
from matplotlib import pyplot as plt

propo_df = pd.DataFrame([sum(results_akhir_df['userid']!=0),sum(results_akhir_df['userid']==0)])
my_labels = "Not Suspected","Order Brushing Suspected"
my_explode = (0, 0.1)
plt.figure(figsize=[8,8])
patches, texts, autotexts = plt.pie(propo_df[0],labels=my_labels,explode=my_explode,autopct='%1.1f%%')
texts[0].set_fontsize(16)
texts[1].set_fontsize(16)
autotexts[0].set_fontsize(16)
autotexts[1].set_fontsize(16)
for autotext in autotexts:
    autotext.set_color('white')
plt.title('Proportion of Order Brushing Shops',fontsize=20)
plt.axis('equal')
plt.show()


In [ ]:
order_brushing_shops_df = pd.DataFrame({'shopid':[],'num_orders':[]})
temp = results_akhir_df['shopid'].loc[results_akhir_df['userid']!=0]
order_brushing_shops_df['shopid'] = order_brushing_shops_df['shopid'].append(temp) 
order_brushing_shops_df['num_orders'] = 0
for shop in order_brushing_shops_df['shopid']:
    order_brushing_shops_df['num_orders'].loc[order_brushing_shops_df['shopid']==shop] = source_df.loc[source_df['shopid']==shop].shape[0]

In [ ]:
non_order_brushing_shops_df = pd.DataFrame({'shopid':[],'num_orders':[]})
temp = results_akhir_df['shopid'].loc[results_akhir_df['userid']==0]
non_order_brushing_shops_df['shopid'] = non_order_brushing_shops_df['shopid'].append(temp) 
non_order_brushing_shops_df['num_orders'] = 0
for shop in non_order_brushing_shops_df['shopid']:
    non_order_brushing_shops_df['num_orders'].loc[non_order_brushing_shops_df['shopid']==shop] = source_df.loc[source_df['shopid']==shop].shape[0]

In [ ]:
order_brushing_shops_df['num_orders'].describe()
plt.figure(figsize=[8,6])
plt.boxplot([order_brushing_shops_df['num_orders'],non_order_brushing_shops_df['num_orders']],showfliers=False)
plt.ylabel('Number of orders',fontsize=16)
plt.xticks([1, 2], ['Suspected', 'Not Suspected'],fontsize=16)